In [ ]:
import os


os.chdir("../")

#
os.getcwd()


'd:\\DA_workspace\\alzheimers_prediction\\src'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from classifier.Mylib import myfuncs
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from xgboost import XGBClassifier


In [ ]:
train_features = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/train_features.pkl"
)
train_target = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/train_target.pkl"
)
val_features = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/val_features.pkl"

)
val_target = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/val_target.pkl"
)

features, target, spliter = myfuncs.get_features_target_spliter_for_CV_train_val(
    train_features, train_target, val_features, val_target
)

param_grid = {"1__n_neighbors": range(5, 20 + 1, 5)}


class NearestNeighborsTransfomer(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=1) -> None:
        super().__init__()
        self.n_neighbors = n_neighbors

    def fit(self, X, y=None):
        self.knn = NearestNeighbors(n_neighbors=self.n_neighbors)

        self.knn.fit(X)

        return self

    def transform(self, X, y=None):
        distance_from_neighbors, index_neighbors = self.knn.kneighbors(X)

        X = pd.DataFrame(distance_from_neighbors)

        return X

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)


pp = Pipeline(steps=[("1", NearestNeighborsTransfomer()), ("2", XGBClassifier())])

gs = GridSearchCV(pp, param_grid, cv=spliter)

gs.fit(features, target)

gs.best_params_


{'1__n_neighbors': 5}

In [ ]:
importance["feature"].tolist()[:3]


['do_get_Genetic_Risk_Factor', 'do_Family_History', 'Age']

In [ ]:
df.shape


(74283, 25)

In [ ]:
df_train, df_test = train_test_split(
    df, stratify=df["Alzheimer’s Diagnosis"], test_size=0.1, random_state=42
)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


data_folder = "../artifacts/data_ingestion"
os.makedirs(data_folder, exist_ok=True)

myfuncs.save_python_object(os.path.join(data_folder, "raw_data.pkl"), df)
myfuncs.save_python_object(os.path.join(data_folder, "train_data.pkl"), df_train)
myfuncs.save_python_object(os.path.join(data_folder, "test_data.pkl"), df_test)
